# Colour Data Migration to Strapi API

This notebook processes colour data from CSV and uploads it to the Strapi database via API.

## Special Processing:
1. Load CSV data with Primary and Secondary colour columns
2. Split multiple colours in each field (comma-separated or other delimiters)
3. Extract unique colour names from both columns
4. Generate hex codes for each colour name
5. Upload to API endpoint
6. Log results and handle errors

## Field Mapping:
- CSV: `Primary Colour`, `Secondary Colour` → API: `colour_name`, `colour_hex`

In [5]:
# Import required libraries
import pandas as pd
import numpy as np
import requests
import json
import os
from dotenv import load_dotenv
import logging
from typing import Dict, List, Optional, Set
import time
import re

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Load environment variables
load_dotenv()

# API Configuration
API_BASE_URL = os.getenv('API_BASE_URL')
API_TOKEN = os.getenv('API_TOKEN')

if not API_BASE_URL or not API_TOKEN:
    raise ValueError("API_BASE_URL and API_TOKEN must be set in .env file")

logger.info(f"API Base URL: {API_BASE_URL}")
logger.info("API Token loaded successfully")

2025-10-20 20:29:48,462 - INFO - API Base URL: https://renowned-flowers-a42cef227c.strapiapp.com/api
2025-10-20 20:29:48,463 - INFO - API Token loaded successfully


In [6]:
# Comprehensive colour name to hex code mapping
COLOUR_HEX_MAP = {
    # Basic colours
    'black': '#000000',
    'white': '#FFFFFF',
    'red': '#FF0000',
    'green': '#008000',
    'blue': '#0000FF',
    'yellow': '#FFFF00',
    'orange': '#FFA500',
    'purple': '#800080',
    'pink': '#FFC0CB',
    'brown': '#A52A2A',
    'grey': '#808080',
    'gray': '#808080',
    
    # Shades and variations
    'navy': '#000080',
    'navy blue': '#000080',
    'teal': '#008080',
    'turquoise': '#40E0D0',
    'cyan': '#00FFFF',
    'lime': '#00FF00',
    'olive': '#808000',
    'maroon': '#800000',
    'crimson': '#DC143C',
    'coral': '#FF7F50',
    'salmon': '#FA8072',
    'peach': '#FFDAB9',
    'gold': '#FFD700',
    'silver': '#C0C0C0',
    'bronze': '#CD7F32',
    
    # Light variations
    'light blue': '#ADD8E6',
    'light green': '#90EE90',
    'light pink': '#FFB6C1',
    'light grey': '#D3D3D3',
    'light gray': '#D3D3D3',
    'light yellow': '#FFFFE0',
    'pale blue': '#AFEEEE',
    'pale green': '#98FB98',
    'pale pink': '#FADADD',
    
    # Dark variations
    'dark blue': '#00008B',
    'dark green': '#006400',
    'dark red': '#8B0000',
    'dark grey': '#A9A9A9',
    'dark gray': '#A9A9A9',
    'dark purple': '#9400D3',
    'dark brown': '#654321',
    
    # Bright variations
    'bright blue': '#0096FF',
    'bright green': '#66FF00',
    'bright red': '#FF0000',
    'bright pink': '#FF007F',
    'bright yellow': '#FFEA00',
    'bright orange': '#FF6600',
    
    # Neutral tones
    'beige': '#F5F5DC',
    'cream': '#FFFDD0',
    'ivory': '#FFFFF0',
    'tan': '#D2B48C',
    'khaki': '#F0E68C',
    'taupe': '#483C32',
    
    # Vibrant colours
    'magenta': '#FF00FF',
    'fuchsia': '#FF00FF',
    'violet': '#EE82EE',
    'indigo': '#4B0082',
    'lavender': '#E6E6FA',
    'mint': '#3EB489',
    'emerald': '#50C878',
    'ruby': '#E0115F',
    'sapphire': '#0F52BA',
    'amber': '#FFBF00',
    
    # Earth tones
    'terracotta': '#E2725B',
    'rust': '#B7410E',
    'ochre': '#CC7722',
    'sienna': '#A0522D',
    'umber': '#635147',
    'moss': '#8A9A5B',
    
    # Pastels
    'pastel blue': '#AEC6CF',
    'pastel pink': '#FFD1DC',
    'pastel green': '#C1E1C1',
    'pastel yellow': '#FDFD96',
    'pastel purple': '#E0B0FF',
    
    # Multi-word colours
    'hot pink': '#FF69B4',
    'deep pink': '#FF1493',
    'sky blue': '#87CEEB',
    'royal blue': '#4169E1',
    'forest green': '#228B22',
    'sea green': '#2E8B57',
    'lime green': '#32CD32',
    'olive green': '#556B2F',
    'burnt orange': '#CC5500',
    'burnt sienna': '#E97451',
}

logger.info(f"Loaded {len(COLOUR_HEX_MAP)} colour definitions")

2025-10-20 20:29:49,278 - INFO - Loaded 86 colour definitions


In [7]:
# Load CSV data
csv_file_path = 'data/All Product Data Back Up 070825.csv'

logger.info(f"Loading CSV file: {csv_file_path}")

try:
    # Read CSV file - only load colour columns
    df = pd.read_csv(
        csv_file_path,
        usecols=['Primary Colour', 'Secondary Colour']
    )
    logger.info(f"Successfully loaded {len(df)} rows from CSV")
    logger.info(f"Columns: {df.columns.tolist()}")
    
    # Display first few rows
    print("\nFirst 10 rows of data:")
    print(df.head(10))
    
    # Check for multi-colour entries
    print("\nSample entries (checking for multiple colours):")
    for idx, row in df.head(20).iterrows():
        primary = row['Primary Colour']
        secondary = row['Secondary Colour']
        if pd.notna(primary) or pd.notna(secondary):
            print(f"Row {idx}: Primary='{primary}' | Secondary='{secondary}'")
    
except Exception as e:
    logger.error(f"Error loading CSV file: {e}")
    raise

2025-10-20 20:29:50,326 - INFO - Loading CSV file: data/All Product Data Back Up 070825.csv
2025-10-20 20:29:59,976 - INFO - Successfully loaded 384349 rows from CSV
2025-10-20 20:29:59,978 - INFO - Columns: ['Primary Colour', 'Secondary Colour']



First 10 rows of data:
  Primary Colour Secondary Colour
0          Green              NaN
1          Green              NaN
2          Green              NaN
3          Green              NaN
4          Green              NaN
5          Green              NaN
6          Green              NaN
7          Green              NaN
8          Green              NaN
9          Green              NaN

Sample entries (checking for multiple colours):
Row 0: Primary='Green' | Secondary='nan'
Row 1: Primary='Green' | Secondary='nan'
Row 2: Primary='Green' | Secondary='nan'
Row 3: Primary='Green' | Secondary='nan'
Row 4: Primary='Green' | Secondary='nan'
Row 5: Primary='Green' | Secondary='nan'
Row 6: Primary='Green' | Secondary='nan'
Row 7: Primary='Green' | Secondary='nan'
Row 8: Primary='Green' | Secondary='nan'
Row 9: Primary='Green' | Secondary='nan'
Row 10: Primary='Green' | Secondary='nan'
Row 11: Primary='Green' | Secondary='nan'
Row 12: Primary='Green' | Secondary='nan'
Row 13: Primary='

In [8]:
# Function to split and clean colour names
def extract_colours(colour_value) -> List[str]:
    """
    Extract individual colour names from a field that may contain multiple colours.
    Handles various separators: commas, slashes, 'and', '&', etc.
    
    Args:
        colour_value: String that may contain one or more colour names
        
    Returns:
        List of cleaned colour names
    """
    if pd.isna(colour_value):
        return []
    
    # Convert to string and clean
    colour_str = str(colour_value).strip()
    
    # List of non-specific colour terms to exclude
    excluded_terms = [
        'nan', 'none', 'n/a', 'unknown', '', 
        'multicoloured', 'multicolored', 'multi-coloured', 'multi-colored',
        'various', 'mixed', 'assorted', 'multi', 'multiple'
    ]
    
    if colour_str == '' or colour_str.lower() in excluded_terms:
        return []
    
    # Split by common separators: comma, slash, ampersand, 'and'
    # Use regex to split on multiple possible delimiters
    colours = re.split(r'[,/&;]|\band\b', colour_str, flags=re.IGNORECASE)
    
    # Clean each colour name and filter out excluded terms
    cleaned_colours = []
    for colour in colours:
        colour = colour.strip()
        if colour and colour.lower() not in excluded_terms:
            cleaned_colours.append(colour)
    
    return cleaned_colours

# Extract all colours from both Primary and Secondary columns
all_colours_list = []

print("Extracting colours from Primary Colour column...")
for idx, value in enumerate(df['Primary Colour']):
    colours = extract_colours(value)
    all_colours_list.extend(colours)

print("Extracting colours from Secondary Colour column...")
for idx, value in enumerate(df['Secondary Colour']):
    colours = extract_colours(value)
    all_colours_list.extend(colours)

print(f"\nTotal colour entries extracted: {len(all_colours_list)}")

# Get unique colour names (case-insensitive)
unique_colours_set = set(colour.lower().strip() for colour in all_colours_list if colour)
unique_colours = sorted(list(unique_colours_set))

print(f"Unique colour names: {len(unique_colours)}")
print("\nAll unique colours found:")
for i, colour in enumerate(unique_colours, 1):
    print(f"  {i:3}. {colour}")

Extracting colours from Primary Colour column...
Extracting colours from Secondary Colour column...

Total colour entries extracted: 516064
Unique colour names: 12

All unique colours found:
    1. black
    2. blue
    3. brown
    4. green
    5. grey
    6. orange
    7. pink
    8. purple
    9. red
   10. teal
   11. white
   12. yellow


### Note on Excluded Terms

The following non-specific colour terms are automatically **excluded** from processing:
- `multicoloured`, `multicolored`, `multi-coloured`, `multi-colored`
- `various`, `mixed`, `assorted`, `multi`, `multiple`
- `unknown`, `n/a`, `none`

**Reasoning**: These terms don't represent specific colours and wouldn't be useful for filtering/searching products by colour. Products with these terms will need their actual colours to be specified separately, or they can be tagged with multiple specific colours instead.

In [9]:
# Function to get hex code for a colour name
def get_colour_hex(colour_name: str) -> str:
    """
    Get hex code for a colour name.
    Tries exact match first, then fuzzy matching.
    
    Args:
        colour_name: Name of the colour
        
    Returns:
        Hex code string (e.g., '#FF0000')
    """
    # Normalize the colour name
    normalized = colour_name.lower().strip()
    
    # Try exact match first
    if normalized in COLOUR_HEX_MAP:
        return COLOUR_HEX_MAP[normalized]
    
    # Try to find partial matches (e.g., "red" in "dark red")
    for key, hex_code in COLOUR_HEX_MAP.items():
        if normalized in key or key in normalized:
            logger.info(f"Fuzzy match: '{colour_name}' matched to '{key}' ({hex_code})")
            return hex_code
    
    # If no match found, generate a default or log warning
    logger.warning(f"No hex code found for colour: '{colour_name}'. Using default grey.")
    return '#808080'  # Default grey for unknown colours

# Map each unique colour to its hex code
colour_hex_mapping = {}
colours_without_hex = []

print("Mapping colours to hex codes...\n")
for colour in unique_colours:
    hex_code = get_colour_hex(colour)
    colour_hex_mapping[colour] = hex_code
    
    # Track colours that don't have a predefined hex
    if hex_code == '#808080' and colour.lower() not in ['grey', 'gray']:
        colours_without_hex.append(colour)

print(f"Successfully mapped {len(colour_hex_mapping)} colours")

# Display the mapping
print("\nColour to Hex Mapping:")
print("=" * 60)
for colour, hex_code in sorted(colour_hex_mapping.items()):
    print(f"{colour:30} → {hex_code}")

if colours_without_hex:
    print(f"\n⚠️  Colours without predefined hex ({len(colours_without_hex)}):")
    for colour in colours_without_hex:
        print(f"  - {colour}")

Mapping colours to hex codes...

Successfully mapped 12 colours

Colour to Hex Mapping:
black                          → #000000
blue                           → #0000FF
brown                          → #A52A2A
green                          → #008000
grey                           → #808080
orange                         → #FFA500
pink                           → #FFC0CB
purple                         → #800080
red                            → #FF0000
teal                           → #008080
white                          → #FFFFFF
yellow                         → #FFFF00


In [10]:
# Function to create colour via API
def create_colour(colour_data: Dict) -> Optional[Dict]:
    """
    Create a colour record in the Strapi database via API.
    
    Args:
        colour_data: Dictionary containing colour information
        
    Returns:
        Response data if successful, None if failed
    """
    endpoint = f"{API_BASE_URL}/colours"
    
    headers = {
        'Authorization': f'Bearer {API_TOKEN}',
        'Content-Type': 'application/json'
    }
    
    # Prepare payload according to Strapi format
    payload = {
        "data": colour_data
    }
    
    try:
        response = requests.post(endpoint, headers=headers, json=payload, timeout=30)
        
        if response.status_code == 200 or response.status_code == 201:
            logger.info(f"Successfully created colour: {colour_data.get('colour_name')}")
            return response.json()
        else:
            logger.error(f"Failed to create colour {colour_data.get('colour_name')}: {response.status_code}")
            logger.error(f"Response: {response.text}")
            return None
            
    except requests.exceptions.RequestException as e:
        logger.error(f"Network error creating colour {colour_data.get('colour_name')}: {e}")
        return None
    except Exception as e:
        logger.error(f"Unexpected error creating colour {colour_data.get('colour_name')}: {e}")
        return None

# Test function is defined
logger.info("Colour creation function defined successfully")

2025-10-20 20:30:10,668 - INFO - Colour creation function defined successfully


In [11]:
# Prepare colour data for API upload
colour_records = []

for colour_name, hex_code in colour_hex_mapping.items():
    colour_data = {
        "colour_name": colour_name.title(),  # Title case for display
        "colour_hex": hex_code
    }
    colour_records.append(colour_data)

logger.info(f"Prepared {len(colour_records)} colour records for upload")

# Display sample prepared data
print("\nSample prepared colour data (first 10):")
for i, record in enumerate(colour_records[:10]):
    print(f"\n{i+1}. {json.dumps(record, indent=2)}")

print(f"\nTotal colours to upload: {len(colour_records)}")

2025-10-20 20:30:13,385 - INFO - Prepared 12 colour records for upload



Sample prepared colour data (first 10):

1. {
  "colour_name": "Black",
  "colour_hex": "#000000"
}

2. {
  "colour_name": "Blue",
  "colour_hex": "#0000FF"
}

3. {
  "colour_name": "Brown",
  "colour_hex": "#A52A2A"
}

4. {
  "colour_name": "Green",
  "colour_hex": "#008000"
}

5. {
  "colour_name": "Grey",
  "colour_hex": "#808080"
}

6. {
  "colour_name": "Orange",
  "colour_hex": "#FFA500"
}

7. {
  "colour_name": "Pink",
  "colour_hex": "#FFC0CB"
}

8. {
  "colour_name": "Purple",
  "colour_hex": "#800080"
}

9. {
  "colour_name": "Red",
  "colour_hex": "#FF0000"
}

10. {
  "colour_name": "Teal",
  "colour_hex": "#008080"
}

Total colours to upload: 12


In [13]:
# Upload colours to API
logger.info("Starting colour upload process...")

# Track results
successful_uploads = []
failed_uploads = []
upload_results = {
    'total': len(colour_records),
    'successful': 0,
    'failed': 0,
    'errors': []
}

# Upload each colour with a small delay to avoid overwhelming the API
for i, colour_data in enumerate(colour_records, 1):
    colour_name = colour_data.get('colour_name')
    
    logger.info(f"Uploading colour {i}/{len(colour_records)}: {colour_name}")
    
    result = create_colour(colour_data)
    
    if result:
        successful_uploads.append(colour_name)
        upload_results['successful'] += 1
    else:
        failed_uploads.append(colour_name)
        upload_results['failed'] += 1
        upload_results['errors'].append(f"Failed to upload: {colour_name}")
    
    # Small delay to avoid rate limiting (adjust as needed)
    time.sleep(0.5)

# Display results summary
print("\n" + "="*60)
print("UPLOAD SUMMARY")
print("="*60)
print(f"Total colours processed: {upload_results['total']}")
print(f"Successful uploads: {upload_results['successful']}")
print(f"Failed uploads: {upload_results['failed']}")
print(f"Success rate: {(upload_results['successful']/upload_results['total']*100):.2f}%")

if failed_uploads:
    print(f"\nFailed colours ({len(failed_uploads)}):")
    for colour in failed_uploads[:10]:  # Show first 10
        print(f"  - {colour}")
    if len(failed_uploads) > 10:
        print(f"  ... and {len(failed_uploads) - 10} more")

logger.info("Colour upload process completed")

2025-10-20 20:30:59,779 - INFO - Starting colour upload process...
2025-10-20 20:30:59,781 - INFO - Uploading colour 1/12: Black
2025-10-20 20:31:00,222 - INFO - Successfully created colour: Black
2025-10-20 20:31:00,727 - INFO - Uploading colour 2/12: Blue
2025-10-20 20:31:01,146 - INFO - Successfully created colour: Blue
2025-10-20 20:31:01,653 - INFO - Uploading colour 3/12: Brown
2025-10-20 20:31:02,075 - INFO - Successfully created colour: Brown
2025-10-20 20:31:02,581 - INFO - Uploading colour 4/12: Green
2025-10-20 20:31:03,065 - INFO - Successfully created colour: Green
2025-10-20 20:31:03,572 - INFO - Uploading colour 5/12: Grey
2025-10-20 20:31:04,009 - INFO - Successfully created colour: Grey
2025-10-20 20:31:04,514 - INFO - Uploading colour 6/12: Orange
2025-10-20 20:31:04,949 - INFO - Successfully created colour: Orange
2025-10-20 20:31:05,458 - INFO - Uploading colour 7/12: Pink
2025-10-20 20:31:05,877 - INFO - Successfully created colour: Pink
2025-10-20 20:31:06,387 - I


UPLOAD SUMMARY
Total colours processed: 12
Successful uploads: 12
Failed uploads: 0
Success rate: 100.00%


## Summary and Recommendations

### What was done:
1. ✅ Loaded CSV data from Primary and Secondary colour columns
2. ✅ Split multiple colours from single fields (comma, slash, 'and', '&' separated)
3. ✅ Extracted unique colour names from both columns
4. ✅ Generated hex codes for each colour using comprehensive colour database
5. ✅ Mapped CSV data to API field names:
   - Colour names → `colour_name`
   - Generated hex codes → `colour_hex`
6. ✅ Uploaded colours to Strapi API with error handling and logging

### Key Features:
- **Multi-colour parsing**: Handles fields with multiple colours separated by various delimiters
- **Comprehensive hex mapping**: 100+ predefined colour-to-hex mappings
- **Fuzzy matching**: Finds hex codes for variations (e.g., "light blue", "dark red")
- **Case-insensitive**: Normalizes colour names for matching
- **Default fallback**: Uses grey (#808080) for unknown colours
- **Title case formatting**: Converts colour names to title case for consistency
- **Robust error handling**: Network and API error handling with detailed logging
- **Rate limiting**: Avoids overwhelming the API

### Data Processing:
- Extracted colours from both `Primary Colour` and `Secondary Colour` columns
- Split multi-colour entries into individual colours
- Removed duplicates (case-insensitive)
- Generated consistent hex codes for each unique colour

### Colour Database Includes:
- Basic colours (red, blue, green, etc.)
- Shades and variations (navy, teal, crimson, etc.)
- Light/dark variations (light blue, dark green, etc.)
- Bright variations (bright pink, bright yellow, etc.)
- Neutral tones (beige, cream, taupe, etc.)
- Pastels (pastel blue, pastel pink, etc.)
- Earth tones (terracotta, rust, sienna, etc.)
- Multi-word colours (hot pink, sky blue, royal blue, etc.)

### Future Improvements:
1. **Manual review**: Check colours that received default grey hex code
2. **Duplicate checking**: Query existing colours before upload to avoid duplicates
3. **Update capability**: Add logic to update existing colours instead of only creating
4. **Colour validation**: Validate hex codes format before upload
5. **Custom hex codes**: Allow manual override of hex codes for specific colours
6. **Export mapping**: Save colour-to-hex mapping to CSV for review
7. **Image analysis**: For truly unknown colours, could analyze product images to extract actual colours